In [1]:
%cd mergekit

/nfs/home/jeanpetityvelosb/project/LLM-merging-for-scientific-communication/mergekit


/nfs/home/jeanpetityvelosb/miniconda3/envs/env/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import yaml

MODEL_NAME = "phi-merge-arithmetic"
yaml_config = """
models:
  - model: microsoft/Phi-3.5-mini-instruct
    parameters:
      weight: 1.0
  - model: microsoft/Phi-3-mini-4k-instruct
    parameters: 
      weight: 0.6
  - model: microsoft/Phi-3-mini-128k-instruct
    parameters: 
      weight: 0.2
base_model: microsoft/Phi-3.5-mini-instruct
merge_method: task_arithmetic
dtype: float16

"""

# Save config as yaml file
# with open('config.yaml', 'w', encoding="utf-8") as f:
#     f.write(yaml_config)

In [4]:
%mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

UsageError: Line magic function `%mergekit-yaml` not found.


In [3]:
from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template
import yaml

username = "yvelos"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [7]:
from huggingface_hub import HfApi
from dotenv import load_dotenv
import os

username = "yvelos"

load_dotenv()

# Defined in the secrets tab in Google Colab
api = HfApi(token=os.environ['HUB_TOKEN'])

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

model-00002-of-00009.safetensors:   0%|          | 0.00/982M [00:00<?, ?B/s]

model-00003-of-00009.safetensors:   0%|          | 0.00/956M [00:00<?, ?B/s]

model-00004-of-00009.safetensors:   0%|          | 0.00/906M [00:00<?, ?B/s]

Upload 10 LFS files:   0%|          | 0/10 [00:00<?, ?it/s]

model-00001-of-00009.safetensors:   0%|          | 0.00/998M [00:00<?, ?B/s]

...

model-00007-of-00009.safetensors:   0%|          | 0.00/906M [00:00<?, ?B/s]

model-00008-of-00009.safetensors:   0%|          | 0.00/906M [00:00<?, ?B/s]

model-00009-of-00009.safetensors:   0%|          | 0.00/176M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yvelos/phi-merge-arithmetic/commit/f25be3775f0297ef691c21d0f78130f02ca34d80', commit_message='Upload folder using huggingface_hub', commit_description='', oid='f25be3775f0297ef691c21d0f78130f02ca34d80', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yvelos/phi-merge-arithmetic', endpoint='https://huggingface.co', repo_type='model', repo_id='yvelos/phi-merge-arithmetic'), pr_revision=None, pr_num=None)